## Извлечение именованных сущеностей

Опрос на сегодня - https://forms.gle/HVfV32cRHCDbNrbK9.

На этом семинаре будем заниматься выделением именованных сущеностей из текстов

Загрузите эмбеддинги c https://nlp.stanford.edu/projects/glove/ или другие, которые вам нравятся и пропишите путь к ним. Можно скачать один файл - https://yadi.sk/i/SFgsTWW6RoP90w.

In [ ]:
import numpy as np

#Идея для улучшения (быстродействия и потребления памяти) - сначала считать корпус, а потом эмбеддинги, и запоминать 
#эмбеддинги только тех слов, которые есть в корпусе
word_embeddings_path = 'glove.6B.50d.txt'
word2idx = {}
word_embeddings = []
embedding_size = None
#Загружаем эмбеддинги
with open(word_embeddings_path, 'r', encoding="utf-8") as f_em:
    for line in f_em:
        split = line.strip().split(" ")
        # Совсем короткие строки пропускаем
        if len(split) <= 2:
            continue
        # Встретив первую подходящую строку, фиксируем размер эмбеддингов
        if embedding_size is None:
            embedding_size = len(split) - 1
            # Также нициализируем эмбеддинги для паддингов и неизвестных слов
            word2idx["PADDING_TOKEN"] = len(word2idx)
            word_embeddings.append(np.zeros(embedding_size))

            word2idx["UNKNOWN_TOKEN"] = len(word2idx)
            word_embeddings.append(np.random.uniform(-0.25, 0.25, embedding_size))
        # После этого все эмбеддинги должны быть одинаковой длины
        if len(split) - 1 != embedding_size:
            continue
        word_embeddings.append(np.asarray(split[1:], dtype='float32'))
        word2idx[split[0]] = len(word2idx)

word_embeddings = np.array(word_embeddings, dtype='float32')


Сделаем впспомогательную функцию, которая определяет тип токена (число и капитализация).

In [ ]:
import tensorflow

#Идея для улучшения - перейдите к шаблонам капитализации (как в Nadeau and Sekine 2007)
case2idx = {'numeric': 0, 'all_lower':1, 'all_upper':2, 'initial_upper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
case_embeddings = np.identity(len(case2idx), dtype='float32')

def get_casing(word, case_lookup):   
    casing = 'other'
    
    num_digits = 0
    for char in word:
        if char.isdigit():
            num_digits += 1
            
    digit_fraction = num_digits / float(len(word))
    
    if word.isdigit(): #Цифра
        casing = 'numeric'
    elif digit_fraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #Все буквы маленькие
        casing = 'all_lower'
    elif word.isupper(): #Все буквы большие
        casing = 'all_upper'
    elif word[0].isupper(): #Первая большая, остальные маленькие
        casing = 'initial_upper'
    elif num_digits > 0:
        casing = 'contains_digit'  
   
    return case_lookup[casing]

Теперь перейдем к обработке собственно нашего обучающего и тестового массива данных в CoNLL формате.

In [ ]:
MAX_COLUMNS = 2
WORD_COL_NUM = 0
LABEL_COL_NUM = 1
def read_file(file_path):
    """
    Получает на вход путь к данным в CoNLL-формате. Выдает массив предложений, разбитых на слова
    :param file_path: путь к корпусу в CoNLL-формате
    :return: corpus_sentences - массив предложений, разбитых на слова
    """
    corpus_sentences = []
    input_sentence = []
    with open(file_path, 'r', encoding='utf-8') as f_in:
        for line in f_in:
            line = line.strip()

            if len(line) == 0 or line[0] == '#':
                if len(input_sentence) > 0:
                    corpus_sentences.append(input_sentence)
                    input_sentence = []
                continue
            if len(line.split('\t')) < MAX_COLUMNS:
                print(line)
                continue
            input_sentence.append(line.split('\t'))

    if len(input_sentence) > 0:
        corpus_sentences.append(input_sentence)

    print(file_path, len(corpus_sentences), "sentences")
    return corpus_sentences

#Пропишите путь к частям корпуса CoNLL-2003
train_path = 'conll.train'
train_sentences = read_file(train_path)

dev_path = 'conll.dev'
dev_sentences = read_file(dev_path)

test_path = 'conll.test'
test_sentences = read_file(test_path)

# Часто у нас есть один корпус, из которого мы должны выделить dev и test часть (по 0.1-0.2 выборки). 
# Напишите код, обрабатывающий такой случай

Считываем все метки классов и добавляем метку для паддингов.

In [ ]:
label_set = set()
label_set.add('PADDING_LABEL')
for dataset in [train_sentences, dev_sentences, test_sentences]:
    for sentence in dataset:
        for token in sentence:
            label = token[LABEL_COL_NUM]
            label_set.add(label)    

# Переводим метки в индексы
label2idx = {}
idx2label = {}
for label in label_set:
    label2idx[label] = len(label2idx)
    
print(label2idx)

Добиваемся того, чтобы предложение всегда начиналось с 1 паддинга и заканчивалось 1 паддингом. Однако, при работе с GPU батчи должны иметь одинаковые размеры. Поэтому имеет смысл разбить предложения на группы так, чтобы все предожения в группе имели одинаковую длину (и значиит предложения одной группы могли попадать в один батч). Простой способ это сделать - добивать все предложения паддингами до одного размера (т. е. иметь одну группу). Большинство предложений короткие, но существуют и очень длинные, поэтому такой способ избыточен.Разумнее иметь более тонкое разбиение на группы - например паддить предложение до следующего 2^n + 1.

In [ ]:
def create_matrices(sentences, word2idx, label2idx, case2idx):   
    
    unknown_idx = word2idx['UNKNOWN_TOKEN']
    padding_casing = case2idx['PADDING_TOKEN']
    padding_idx = word2idx['PADDING_TOKEN'] 
    padding_label = label2idx['PADDING_LABEL']  
    
    dataset = []
    total_tokens = 0
    unknown_tokens = 0
    for sentence in sentences:
        
        # Индекс первого не паддинга в предложении с паддингами
        proper_sentence_start = 1

        word_indices = np.array([padding_idx] * (len(sentence) + 2))
        case_indices = np.array([padding_casing] * (len(sentence) + 2))
        label_indices = np.array([padding_label] * (len(sentence) + 2))
        
        

        for pos_in_sentence, word in enumerate(sentence):

            token_unknown, word_idx, case_idx = get_token_indices(word, word2idx, case2idx, unknown_idx)

            pos_in_padded_sentence = pos_in_sentence + proper_sentence_start
            word_indices[pos_in_padded_sentence] = word_idx
            case_indices[pos_in_padded_sentence] = case_idx
            label_indices[pos_in_padded_sentence] = label2idx[word[LABEL_COL_NUM]]

            # Хотим вычислить процент словоформ, не покрываемых эмбеддингами
            total_tokens += 1
            if token_unknown:
                unknown_tokens += 1

        # Все данные для одного предложения помещаем в один массив
        dataset.append([word_indices, case_indices, label_indices])
        
    percent = 0.0
    if total_tokens != 0:
        percent = float(unknown_tokens) / total_tokens * 100
    print("{} tokens, {} unknown, {:.3}%".format(total_tokens, unknown_tokens, percent ))
    return dataset

def get_token_indices(token, word2idx, case2idx, unknown_idx):

    token_unknown = False
    # Элемент предложения - несколько колонок, словоформа, при этом, в первой колонке
    word = token[WORD_COL_NUM]
    # Ищем слово в словаре эмбеддингов, если не нашли, то ищем слово со снятой капитализацией,
    # если нет - считаем слово неизвестным
    if word2idx.get(word) is not None:
        word_idx = word2idx[word]
    elif word2idx.get(word.lower()) is not None:
        word_idx = word2idx[word.lower()]
    else:
        word_idx = unknown_idx
        token_unknown = True

    case_idx = get_casing(word, case2idx)
    return token_unknown, word_idx, case_idx

train_data = create_matrices(train_sentences, word2idx, label2idx, case2idx)
dev_data = create_matrices(dev_sentences, word2idx, label2idx, case2idx)
test_data = create_matrices(test_sentences, word2idx, label2idx, case2idx)

for sentence in train_datas[:5]:
    print(sentence)

In [ ]:
#Идея для улучшения - использовать символьные признаки - брать первые  или последние (а лучше и то, и другое) k сиволов
#каждого токена и пропускать через рекуррентный или сверточный слой. 
#Потом получившееся конкатенировать с остальными признакаи в merged_embeddings
# Про эту идею можно почитать здесь https://arxiv.org/pdf/1603.01360v1.pdf 


from keras.layers import Embedding, LSTM, Dense, TimeDistributed, Dropout, Bidirectional, Input, concatenate
from keras.models import Model
from keras.optimizers import Adam
# На боевом применении должно быть 100 или больше
SENTENCE_LSTM_DIM = 10

n_out = len(label2idx)

tokens_input = Input(dtype='int32', shape=(None,), name='tokens_input')
tokens_embedding_layer = Embedding(input_dim=word_embeddings.shape[0], 
                                   output_dim=word_embeddings.shape[1],
                                   weights=[word_embeddings], trainable=False, 
                                   name='tokens_embeddings')
tokens = tokens_embedding_layer(tokens_input)


casing_input = Input(dtype='int32', shape=(None,), name='casing_input')
casing_embedding_layer = Embedding(input_dim=case_embeddings.shape[0], 
                                   output_dim=case_embeddings.shape[1],
                                   weights=[case_embeddings], trainable=True, 
                                   name='casing_embeddings')
casing = casing_embedding_layer(casing_input)

merged_embeddings = concatenate([tokens, casing], name='merged_embeddings')
for_lstm = Dropout(0.2)(merged_embeddings)
# Если настроите работу на GPU, лучше использовать implementation=2
blstm = Bidirectional(LSTM(SENTENCE_LSTM_DIM, return_sequences=True, implementation=0), 
                      name='blstm')(for_lstm)
#Здесь имеет смысл попробовать другие варианиты - GRU, свертки, а также несколько подряд 
#идущих слоев LSTM или GRU
#result = Conv1d(n_out,1, activation='softmax', name='result'))(blstm
result = TimeDistributed(Dense(n_out,activation='softmax', name='result'))(blstm)

model = Model(inputs=[tokens_input, casing_input], outputs=result)

# default lr = 0.001, beta_1=0.9
adam = Adam(lr=0.001, beta_1=0.9)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
model.summary()

In [ ]:
import random
import time
# Если вам удалось разбить предложения на группы одинаковой длины здесь имеет смысл выдвавть батчи из предложений одинаковой 
# длины размера не более BATCH_SIZE (подобранного так, чтобы батч влезал в память вашей видеокарты)
def iterate_minibatches(dataset):   
    for sentence in dataset:
        tokens, casing, labels = sentence     
            
        labels = np.expand_dims(labels, -1) 
        yield np.asarray([tokens]), np.asarray([casing]), np.asarray([labels])

# Здесь, соответсвенно, тоже нужно адаптировать код для батчей разной длины        
def tag_dataset(dataset):
    predicted_labels = []
    correct_labels = []
    for tokens, casing, labels in dataset:
        pred = model.predict_on_batch([np.asarray([tokens]), np.asarray([casing])])[0]
        pred_labels = [el.tolist().index(max(el)) for el in pred]
        predicted_labels.append(pred_labels)
        correct_labels.append(labels)
        #print(predicted_labels, correct_labels)
    return predicted_labels, correct_labels

#В качестве метрики лучше посчитать f-меру по "склеенным" сущностям. Можно реализовать склейку как здесь 
# https://github.com/mit-nlp/MITIE/blob/master/tools/ner_conll/conlleval, можно придумать свой разумный способ
def compute_accuracy(predictions, correct, padding_label):
    """
    Получаем на вход предсказанные и истинные метки, считаем точность на них. Паддинги при подсчете не учитываем
    """
    total_tokens = 0
    guessed_tokens = 0
    for guessed_sentence, correct_sentence in zip(predictions, correct):
        #print(guessed_sentence, correct_sentence)
        assert (len(guessed_sentence) == len(correct_sentence)), "Guessed and correct sentences do not match"
        for j in range(len(guessed_sentence)):
            if correct_sentence[j] != padding_label:
                total_tokens += 1
                if guessed_sentence[j] == correct_sentence[j]:
                    guessed_tokens += 1

    if total_tokens == 0:
        return float(0)
    else:
        accuracy = float(guessed_tokens) / total_tokens
        return accuracy

        
number_of_epochs = 1
print("%d epochs" % number_of_epochs)

print("%d train sentences" % len(train_data))
print("%d dev sentences" % len(dev_data))
print("%d test sentences" % len(test_data))

padding_label = label2idx['PADDING_LABEL']

for epoch in range(number_of_epochs):    
    print("--------- Epoch %d -----------" % epoch)
    random.shuffle(train_data)
    
    start_time = time.time()    
    for batch in iterate_minibatches(train_data):
        #print(batch)
        tokens, casing, labels = batch       
        model.train_on_batch([tokens, casing], labels)   
    print("%.2f sec for training" % (time.time() - start_time))
               
    #Train Dataset       
    start_time = time.time()  
    print("================================== Train Data ==================================")
    predicted_labels, correct_labels = tag_dataset(train_data)        
    accuracy = compute_accuracy(predicted_labels, correct_labels, padding_label)
    print("Accuracy = ", accuracy)

    #Dev Dataset 
    print("================================== Dev Data: ==================================")
    predicted_labels, correct_labels = tag_dataset(dev_data)  
    accuracy = compute_accuracy(predicted_labels, correct_labels, padding_label)
    print("Accuracy = ", accuracy)


    #Test Dataset 
    #state-of-the-art f-мера~0.91 на test
    print("================================== Test Data: ==================================")
    predicted_labels, correct_labels = tag_dataset(test_data)  
    accuracy = compute_accuracy(predicted_labels, correct_labels, padding_label)
    print("Accuracy = ", accuracy)

        
    print("%.2f sec for evaluation" % (time.time() - start_time))


In [ ]:
# Перечислим еще раз все идеи для улучшения, которые можно реализовать
# 1. Ограничить работу с эмбеддингами только теми словами, которые встречаются в корпусе. Это ускоряет быстродействие,
# но, разумеется, не применимо, если применение сети предполагается на неизвестных данных
# 2. Сделать трейн-тест сплит удобного вида)
# 3. Перейти от 8 типов капитализации к шаблонам общего вида
# 4. Паддить предложения не по одному паддингу с каэдой стороны, а до следующего 2^n + 1
# 5. При условии реализации пункта 4 добиться адекватной работы с батчами - добиться, чтобы iterate_minibatches выдавала
# батчи из BATCH_SIZE предложений одинакового размера(имеет смысл только если есть желание настроить работу сетки на GPU)
# 6. Добавить в сеть символьную часть - подавать эмбеддинги CHAR_SIZE первых символов каждого токена LSTM (или CNN), 
# результат применения которого конкатенировать со словоформенными эмбеддингами. То же самое сделать с CHAR_SIZE
# последними символами
# 7. Провести другие эксперименты с архитектурой - сделать LSTM многослойным, заменить на GRU или CNN и т. п.
# 8. Добавить в эвалюейшн подсчет ф-меры со "склейкой" сущностей
# Если сделать все получится близакая к state-of-the-art сетка - на test f-мера >= 0.905